In [1]:
import pandas as pd
import numpy as np
import json

# Start by loading the population data

In [2]:
raw_pop_data = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\NY_population_by_zip_code.csv', header=1)

In [3]:
raw_pop_data.head()

,Id,zipcode,Geography,population,Margin of Error; Total
0,8600000US06390,6390,ZCTA5 06390,230,139
1,8600000US07421,7421,ZCTA5 07421,7351,509
2,8600000US10001,10001,ZCTA5 10001,23947,1738
3,8600000US10002,10002,ZCTA5 10002,77925,2031
4,8600000US10003,10003,ZCTA5 10003,55035,1789


# Get rid of unneccessary columns

In [4]:
pop_data = raw_pop_data.drop(['Id', 'Geography', 'Margin of Error; Total'], axis=1)
pop_data.columns = ['postalcode', 'population']
pop_data.head()

,postalcode,population
0,6390,230
1,7421,7351
2,10001,23947
3,10002,77925
4,10003,55035


# Now load income data

In [5]:
raw_income_data = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\NY_med_income.csv', header=1)

# Get rid of unnecessary columns

In [6]:
income_data = raw_income_data[['Id2', 'Median income (dollars); Estimate; Households']]
income_data.columns = ['postalcode', 'med_income']
income_data.head()

,postalcode,med_income
0,6390,150703
1,7421,90412
2,10001,85221
3,10002,35449
4,10003,104441


# Merge population and income data

In [7]:
ny_pop_inc = pop_data.merge(income_data, on='postalcode')

In [8]:
ny_pop_inc.head()

,postalcode,population,med_income
0,6390,230,150703
1,7421,7351,90412
2,10001,23947,85221
3,10002,77925,35449
4,10003,55035,104441


# Load restaurant data

In [9]:
ny_data = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\NYC_Restaurant_Inspection_Results.csv', header=0)

In [10]:
ny_data = ny_data.drop_duplicates(subset=['DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE', 'CUISINE DESCRIPTION'])

In [11]:
ny_data.head()

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,VIOLATION DESCRIPTION,SCORE,GRADE
0,UNCLE LOUIE G’S ITALIAN ICES & ICE CREAM,Brooklyn,741,UNION STREET,11215.0,7186236668,"Ice Cream, Gelato, Yogurt, Ices",Evidence of mice or live mice present in facil...,22.0,B
1,ARABICA HOOKAH LOUNGE & RESTAURANT INC,Staten Island,4162,VICTORY BLVD,10314.0,7189828888,Middle Eastern,Personal cleanliness inadequate. Outer garment...,12.0,A
2,BOEUF & BUN,Brooklyn,271,KINGSTON AVE,11213.0,7182218900,Jewish/Kosher,Facility not vermin proof. Harborage or condit...,18.0,NaN
3,EL TRI MX RESTAURANT & BAR CORP,Queens,9912,NORTHERN BLVD,11368.0,7184249600,Mexican,“Choking first aid” poster not posted. “Alcoho...,NaN,NaN
4,BAMBOO GARDEN CHINESE RESTAURANT,Queens,14610,LIBERTY AVENUE,11435.0,7182061339,Chinese,Facility not vermin proof. Harborage or condit...,18.0,NaN


In [12]:
len(ny_data['ZIPCODE'].unique())

228

# Group by zip code and calculate the proportion of each cuisine type

In [13]:
cuisine_prop = ny_data.groupby(['ZIPCODE','CUISINE DESCRIPTION']).size() / ny_data.groupby('ZIPCODE').size()
cuisine_prop = cuisine_prop.unstack(level=-1, fill_value=0)
cuisine_prop.reset_index(inplace=True)
cuisine_prop = cuisine_prop.rename(columns = {'ZIPCODE': 'postalcode'})
cuisine_prop.head(10)
restaurants = cuisine_prop
restaurants.head(10)

CUISINE DESCRIPTION,postalcode,Afghan,African,American,Armenian,Asian,Australian,Bagels/Pretzels,Bakery,Bangladeshi,...,Soups & Sandwiches,Southwestern,Spanish,Steak,Tapas,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese/Cambodian/Malaysia
0,10000.0,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,10001.0,0.000000,0.003839,0.312860,0.0,0.019194,0.001919,0.009597,0.009597,0.000000,...,0.000000,0.0,0.015355,0.005758,0.001919,0.003839,0.001919,0.0,0.003839,0.000000
2,10002.0,0.000000,0.000000,0.197318,0.0,0.036398,0.005747,0.001916,0.057471,0.001916,...,0.000000,0.0,0.013410,0.000000,0.003831,0.001916,0.017241,0.0,0.007663,0.009579
3,10003.0,0.001471,0.000000,0.220588,0.0,0.022059,0.001471,0.002941,0.013235,0.002941,...,0.001471,0.0,0.002941,0.001471,0.002941,0.004412,0.019118,0.0,0.016176,0.004412
4,10004.0,0.000000,0.000000,0.297297,0.0,0.013514,0.000000,0.054054,0.006757,0.000000,...,0.006757,0.0,0.000000,0.006757,0.000000,0.000000,0.006757,0.0,0.000000,0.000000
5,10005.0,0.000000,0.000000,0.357143,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,10006.0,0.000000,0.000000,0.431373,0.0,0.019608,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.019608,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,10007.0,0.000000,0.000000,0.318841,0.0,0.014493,0.000000,0.007246,0.007246,0.007246,...,0.000000,0.0,0.007246,0.014493,0.000000,0.021739,0.000000,0.0,0.007246,0.000000
8,10009.0,0.000000,0.000000,0.279461,0.0,0.037037,0.000000,0.006734,0.013468,0.000000,...,0.000000,0.0,0.013468,0.000000,0.006734,0.003367,0.013468,0.0,0.020202,0.006734
9,10010.0,0.000000,0.000000,0.317829,0.0,0.011628,0.000000,0.015504,0.007752,0.000000,...,0.003876,0.0,0.003876,0.003876,0.007752,0.011628,0.019380,0.0,0.007752,0.011628


In [14]:
len(restaurants['postalcode'].unique())

227

# Load the geojson zip code data

In [15]:
filepath = r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\nyc_zip_code_tabulation_areas_polygons.geojson'

with open(filepath) as json_file:
    json_data = json.load(json_file)

## Let's find out which zip codes are in the geojson file

In [16]:
json_zips = []

for f in json_data['features']:
    json_zips.append(f['properties']['postalcode'])

In [17]:
len(json_zips)

262

In [18]:
len(set(json_zips))

247

# There are more zip codes in the geojson file than in the restaurant file. Also there are duplicates in the geojson.

## Let's find out which zip codes differ between the two.

In [19]:
restaurants_set = set(restaurants['postalcode'])

In [20]:
for i in range(0, len(json_zips)): 
    json_zips[i] = int(json_zips[i])

json_set = set(json_zips)

In [21]:
restaurants_not_json = restaurants_set.difference(json_set)

json_not_restaurants = json_set.difference(restaurants_set)

In [22]:
restaurants_not_json

{10000.0, 10179.0, 11241.0, 11242.0, 11249.0, 11550.0, 12345.0}

In [23]:
json_not_restaurants

{83,
 10043,
 10045,
 10047,
 10081,
 10096,
 10097,
 10104,
 10110,
 10120,
 10122,
 10152,
 10162,
 10166,
 10175,
 10196,
 10203,
 10259,
 10260,
 10265,
 10270,
 10275,
 10278,
 10286,
 11096,
 11359,
 11451}

# The zip codes do not match exactly so we need to make them match.

## First let's remove duplicates from the json

In [24]:
data = json_data['features']

unique_elements = { each['properties']['postalcode'] : each for each in data }.values()
len(unique_elements)

247

In [25]:
json_data['features'] = list(unique_elements)

In [26]:
len(json_data['features'])

247

# Now I delete rows from the restaurants data that is not in the json.

In [27]:
restaurants = restaurants[~restaurants['postalcode'].isin(list(restaurants_not_json))]

In [28]:
len(restaurants)

220

# Now I delete rows from the json that are not in the restaurants data.

In [29]:
new_features = []
for element in json_data['features']:
    if int(element['properties']['postalcode']) not in list(json_not_restaurants):
        new_features.append(element)

In [30]:
json_data['features'] = new_features

In [31]:
len(json_data['features'])

220

# Now merge the restaurants data with other data useful for mapping

In [32]:
# Convert the median income variable to numeric
ny_pop_inc['med_income'] = pd.to_numeric(ny_pop_inc['med_income'], errors='coerce')
# Convert the postalcode variable to int
restaurants['postalcode'] = restaurants['postalcode'].astype('int')
# Merge the population and income data with the restaurant data
restaurants = restaurants.merge(ny_pop_inc, how='left', on='postalcode')

C:\Users\Evan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [33]:
# Calculate the total number of restaurants in each zipcode and merge into the dataframe
total_restaurants = ny_data.groupby('ZIPCODE').size()
total_restaurants = pd.DataFrame(data = {'postalcode':total_restaurants.index, 'total_restaurants':total_restaurants.values}, dtype=np.int64)
restaurants = restaurants.merge(total_restaurants, how='left', on='postalcode')

In [34]:
# Calculate number of restaurants per capita
restaurants['RPC'] = restaurants['total_restaurants'] / restaurants['population']

In [35]:
restaurants.head()

,postalcode,Afghan,African,American,Armenian,Asian,Australian,Bagels/Pretzels,Bakery,Bangladeshi,...,Tapas,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese/Cambodian/Malaysia,population,med_income,total_restaurants,RPC
0,10001,0.000000,0.003839,0.312860,0.0,0.019194,0.001919,0.009597,0.009597,0.000000,...,0.001919,0.003839,0.001919,0.0,0.003839,0.000000,23947.0,85221.0,521,0.021756
1,10002,0.000000,0.000000,0.197318,0.0,0.036398,0.005747,0.001916,0.057471,0.001916,...,0.003831,0.001916,0.017241,0.0,0.007663,0.009579,77925.0,35449.0,522,0.006699
2,10003,0.001471,0.000000,0.220588,0.0,0.022059,0.001471,0.002941,0.013235,0.002941,...,0.002941,0.004412,0.019118,0.0,0.016176,0.004412,55035.0,104441.0,680,0.012356
3,10004,0.000000,0.000000,0.297297,0.0,0.013514,0.000000,0.054054,0.006757,0.000000,...,0.000000,0.000000,0.006757,0.0,0.000000,0.000000,2990.0,140450.0,148,0.049498
4,10005,0.000000,0.000000,0.357143,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,8680.0,154213.0,70,0.008065


# Output the data for analysis and further cleaning

In [36]:
with open(r'C:\Users\Evan\Desktop\Capstone 2\Data\Intermediate Data\JSONs\nyc_zip_codes.geojson', 'w') as outfile:
    json.dump(json_data, outfile)

In [37]:
restaurants.to_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Intermediate Data\CSVs\restaurants.csv')